In [59]:
import pandas as pd

In [177]:
df = pd.read_csv(rf'/Users/danielchung/Downloads/TIP PARAM - Sheet1.csv')

# Convert the 'your_column' from float to int
df.dropna(inplace=True)
df['AP_INV'] = df['AP_INV'].astype(int)
df['PRICE_INDEX_NUM'] = df['PRICE_INDEX_NUM'].astype(int)

pivot_df = df.pivot_table(
    index=["PRICE_INDEX", "PRICE_INDEX_NUM", "ROUTE", "INVENTORY_CARRIER_CD"],
    columns=["AP_INV", "P_OPTION"],
    values=["P_PCT"],
    aggfunc='max'
)

pivot_df2 = df.pivot_table(
    index=["PRICE_INDEX", "PRICE_INDEX_NUM", "ROUTE", "INVENTORY_CARRIER_CD"],
    columns=["AP_INV"],
    values=["BUCKETS"],
    aggfunc='max'
)

# Flatten the multi-level column index by joining the levels with an underscore
# This also trims any excess padding that might cause additional underscores
pivot_df.columns = ['_'.join(str(level).strip() for level in col).rstrip('_') for col in pivot_df.columns.values]
pivot_df2.columns = ['_'.join(str(level).strip() for level in col).rstrip('_') for col in pivot_df2.columns.values]

merged_df = pd.merge(pivot_df, pivot_df2, on=["PRICE_INDEX", "PRICE_INDEX_NUM", "ROUTE", "INVENTORY_CARRIER_CD"], how="left")
# Reset the index to make the DataFrame flat
merged_df = merged_df.reset_index()

merged_df.sort_values(by = 'PRICE_INDEX_NUM', inplace=True)

merged_df = merged_df[['PRICE_INDEX', 'PRICE_INDEX_NUM', 'ROUTE', 'INVENTORY_CARRIER_CD', 'P_PCT_180_P1', 'P_PCT_180_P2', 'BUCKETS_180', 'P_PCT_120_P1', 'P_PCT_120_P2', 'BUCKETS_120' ,'P_PCT_100_P1', 'P_PCT_100_P2', 'BUCKETS_100']]

In [142]:
# Your starting point
initial_value = 0.5
df = pd.DataFrame({
     'Index': [1, 2, 3, 4, 5],
     'P1': [20,20,20,20,20],
     'BUCKETS': [53, 4, 20, 12, 30]
 })

cumulative_sum = 0
adjusted = False
for index, row in df.iterrows():
    if not adjusted:
        cumulative_sum += row['BUCKETS']
        if cumulative_sum > 100:
            excess = cumulative_sum - 100
            df.at[index, 'BUCKETS'] -= excess
            adjusted = True
    else:
        df.at[index, 'BUCKETS'] = 0

# Set the 'P1' for the first index
df.at[0, 'P1'] = initial_value - (initial_value * (df.at[0, 'BUCKETS'] / 100))

# Calculate the 'P1' for the rest of the rows
for i in range(1, len(df)):
    # Calculate the new value for 'P1'
    new_value = df.at[i - 1, 'P1'] - (initial_value * (df.at[i, 'BUCKETS'] / 100))
    df.at[i, 'P1'] = new_value.round(2)


df.at[len(df) - 1, 'P1'] = 0

/var/folders/tp/nvf10t5j7rj5_9gnjk0590jr0000gn/T/ipykernel_72466/2082182411.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.235' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[0, 'P1'] = initial_value - (initial_value * (df.at[0, 'BUCKETS'] / 100))


In [143]:
df

,Index,P1,BUCKETS
0,1,0.235,53
1,2,0.220,4
2,3,0.120,20
3,4,0.060,12
4,5,0.000,11


In [242]:
def apply_buckets(df, buckets, p1_to_adjust, initial_p1, p2_to_adjust, initial_p2):

    df = df[df['INVENTORY_CARRIER_CD']=='BA']
    df.sort_values(by='PRICE_INDEX_NUM', inplace=True)
    df = df.reset_index(drop=True)

    for colb in buckets:
        adjusted = False
        cumulative_sum = 0
        for index_price_num, row in df.iterrows():
            if not adjusted:
                cumulative_sum += row[colb]
                if cumulative_sum > 100:
                    excess = cumulative_sum - 100
                    df.at[index_price_num, colb] -= excess
                    adjusted = True
            else:
                df.at[index_price_num, colb] = 0

        for colp in p1_to_adjust:
            # Set the 'P1' for the first index
            df.at[0, colp] = initial_p1 - (initial_p1 * (df.at[0, colb] / 100)).round(2)

            # Calculate the 'P1' for the rest of the rows
            for i in range(1, len(df)):
                # Calculate the new value for 'P1'
                new_value = df.at[i - 1, colp] - (initial_p1 * (df.at[i, colb] / 100))
                df.at[i, colp] = round(max(new_value, 0), 2)
            
            df.at[len(df) - 1, colp] = 0

        for colp2 in p2_to_adjust:
            # Set the 'P1' for the first index
            df.at[0, colp2] = initial_p2 - (initial_p2 * (df.at[0, colb] / 100)).round(2)

            # Calculate the 'P1' for the rest of the rows
            for i in range(1, len(df)):
                # Calculate the new value for 'P1'
                new_value = df.at[i - 1, colp2] - (initial_p2 * (df.at[i, colb] / 100))
                df.at[i, colp2] = round(max(new_value, 0), 2)
            
            df.at[len(df) - 1, colp2] = 0

    return df, adjusted

In [236]:
p1_to_adjust = ['P_PCT_180_P1', 'P_PCT_120_P1', 'P_PCT_100_P1']
p2_to_adjust = ['P_PCT_180_P2', 'P_PCT_120_P2', 'P_PCT_100_P2']
buckets = ['BUCKETS_180', 'BUCKETS_120', 'BUCKETS_100']
initial_p1 = 0.5
initial_p2 = 0.4

In [224]:
merged_df = merged_df[merged_df['INVENTORY_CARRIER_CD']=='BA']
merged_df.sort_values(by='PRICE_INDEX_NUM', inplace=True)
merged_df = merged_df.reset_index(drop=True)

In [221]:
df = merged_df.copy()

p1_to_adjust = ['P_PCT_180_P1', 'P_PCT_120_P1', 'P_PCT_100_P1']
initial_p1 = 0.4
buckets = ['BUCKETS_180', 'BUCKETS_120', 'BUCKETS_100']
for colb in buckets:
    adjusted = False
    cumulative_sum = 0
    for index_price_num, row in df.iterrows():
        if not adjusted:
            cumulative_sum += row[colb]
            if cumulative_sum > 100:
                excess = cumulative_sum - 100
                df.at[index_price_num, colb] -= excess
                adjusted = True
        else:
            df.at[index_price_num, colb] = 0
        
    df.at[0, 'P_PCT_180_P1'] = initial_p1 - (initial_p1 * (df.at[0, 'BUCKETS_180'] / 100)).round(2)

    # Calculate the 'P1' for the rest of the rows
    for i in range(1, len(df)):
        # Calculate the new value for 'P1'
        new_value = df.at[i - 1, 'P_PCT_180_P1'] - (initial_p1 * (df.at[i, 'BUCKETS_180'] / 100))
        print(new_value)
        df.at[i, 'P_PCT_180_P1'] = round(max(new_value, 0), 2)
    


0.276
0.22000000000000003
0.212
0.146
0.09399999999999999
0.03
-0.0020000000000000018
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.276
0.22000000000000003
0.212
0.146
0.09399999999999999
0.03
-0.0020000000000000018
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.276
0.22000000000000003
0.212
0.146
0.09399999999999999
0.03
-0.0020000000000000018
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [222]:
df

,PRICE_INDEX,PRICE_INDEX_NUM,ROUTE,INVENTORY_CARRIER_CD,P_PCT_180_P1,P_PCT_180_P2,BUCKETS_180,P_PCT_120_P1,P_PCT_120_P2,BUCKETS_120,P_PCT_100_P1,P_PCT_100_P2,BUCKETS_100
0,I0,0,LHRDEN,BA,0.32,0.32,19,9.00,9.00,19,9.00,9.00,18
1,I1,1,LHRDEN,BA,0.28,0.27,11,9.00,9.00,18,9.00,9.00,12
2,I2,2,LHRDEN,BA,0.22,0.20,15,9.00,9.00,18,9.00,9.00,12
3,I3,3,LHRDEN,BA,0.21,0.16,2,9.00,9.00,13,9.00,9.00,14
4,I4,4,LHRDEN,BA,0.15,0.13,16,9.00,9.00,16,9.00,9.00,19
5,I5,5,LHRDEN,BA,0.09,0.11,14,0.21,0.32,16,0.21,0.32,20
6,I6,6,LHRDEN,BA,0.03,0.09,15,0.18,0.27,0,0.21,0.31,4
7,I7,7,LHRDEN,BA,0.00,0.07,8,0.14,0.21,0,0.21,0.31,1
8,I8,8,LHRDEN,BA,0.00,0.06,0,0.08,0.13,0,0.20,0.30,0
9,I9,9,LHRDEN,BA,0.00,0.06,0,0.06,0.09,0,0.20,0.30,0


In [243]:
df, changes_made = apply_buckets(merged_df, buckets, p1_to_adjust, initial_p1, p2_to_adjust, initial_p2)

In [245]:
df

,PRICE_INDEX,PRICE_INDEX_NUM,ROUTE,INVENTORY_CARRIER_CD,P_PCT_180_P1,P_PCT_180_P2,BUCKETS_180,P_PCT_120_P1,P_PCT_120_P2,BUCKETS_120,P_PCT_100_P1,P_PCT_100_P2,BUCKETS_100
0,I0,0,LHRDEN,BA,0.41,0.33,19,0.41,0.33,19,0.41,0.33,18
1,I1,1,LHRDEN,BA,0.35,0.28,11,0.35,0.28,18,0.35,0.28,12
2,I2,2,LHRDEN,BA,0.29,0.23,15,0.29,0.23,18,0.29,0.23,12
3,I3,3,LHRDEN,BA,0.22,0.17,2,0.22,0.17,13,0.22,0.17,14
4,I4,4,LHRDEN,BA,0.12,0.09,16,0.12,0.09,16,0.12,0.09,19
5,I5,5,LHRDEN,BA,0.02,0.01,14,0.02,0.01,16,0.02,0.01,20
6,I6,6,LHRDEN,BA,0.00,0.00,15,0.00,0.00,0,0.00,0.00,4
7,I7,7,LHRDEN,BA,0.00,0.00,8,0.00,0.00,0,0.00,0.00,1
8,I8,8,LHRDEN,BA,0.00,0.00,0,0.00,0.00,0,0.00,0.00,0
9,I9,9,LHRDEN,BA,0.00,0.00,0,0.00,0.00,0,0.00,0.00,0


In [246]:
index_columns = ['PRICE_INDEX', 'PRICE_INDEX_NUM', 'ROUTE', 'INVENTORY_CARRIER_CD']
df.set_index(index_columns, inplace=True)

In [247]:
df

,,,,P_PCT_180_P1,P_PCT_180_P2,BUCKETS_180,P_PCT_120_P1,P_PCT_120_P2,BUCKETS_120,P_PCT_100_P1,P_PCT_100_P2,BUCKETS_100
PRICE_INDEX,PRICE_INDEX_NUM,ROUTE,INVENTORY_CARRIER_CD,,,,,,,,,
I0,0,LHRDEN,BA,0.41,0.33,19,0.41,0.33,19,0.41,0.33,18
I1,1,LHRDEN,BA,0.35,0.28,11,0.35,0.28,18,0.35,0.28,12
I2,2,LHRDEN,BA,0.29,0.23,15,0.29,0.23,18,0.29,0.23,12
I3,3,LHRDEN,BA,0.22,0.17,2,0.22,0.17,13,0.22,0.17,14
I4,4,LHRDEN,BA,0.12,0.09,16,0.12,0.09,16,0.12,0.09,19
I5,5,LHRDEN,BA,0.02,0.01,14,0.02,0.01,16,0.02,0.01,20
I6,6,LHRDEN,BA,0.00,0.00,15,0.00,0.00,0,0.00,0.00,4
I7,7,LHRDEN,BA,0.00,0.00,8,0.00,0.00,0,0.00,0.00,1
I8,8,LHRDEN,BA,0.00,0.00,0,0.00,0.00,0,0.00,0.00,0


In [89]:
df = pd.DataFrame({
     'Index': [1, 2, 3, 4, 5],
     'P1': [20,20,20,20,20],
     'BUCKETS': [20, 1003, 50, 40, 20]
 })





In [90]:
df

,Index,P1,BUCKETS
0,1,20,20
1,2,20,80
2,3,20,0
3,4,20,0
4,5,20,0


In [83]:
df

,Index,P1,BUCKETS
0,1,20,20
1,2,20,10
2,3,20,60
3,4,20,40
4,5,20,60


In [ ]:
# Ensure bucket sizes equal to 100
for i in range(1, len(df)):
    # Calculate the new value for 'P1'
    new_value = df.at[i - 1, 'P1'] - (initial_value * (df.at[i, 'BUCKETS'] / 100))
    df.at[i, 'P1'] = new_value

df['P1'] = df['P1'].round(2)

In [79]:
df

,Index,P1,BUCKETS
0,1,0.40,20
1,2,0.35,10
2,3,0.30,10
3,4,0.10,40
4,5,0.00,20
